In [5]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
# Define the model
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [7]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
  W_sm_exp = dy.parameter(W_sm)
  b_sm_exp = dy.parameter(b_sm)
  return W_sm_exp * cbow + b_sm_exp

In [8]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5345, time=0.65s
iter 0: test acc=0.3543
iter 1: train loss/sent=1.1574, time=0.60s
iter 1: test acc=0.4122
iter 2: train loss/sent=0.8836, time=0.61s
iter 2: test acc=0.3882
iter 3: train loss/sent=0.6510, time=0.58s
iter 3: test acc=0.3738
iter 4: train loss/sent=0.4698, time=0.58s
iter 4: test acc=0.3729
iter 5: train loss/sent=0.3364, time=0.71s
iter 5: test acc=0.3719
iter 6: train loss/sent=0.2461, time=0.61s
iter 6: test acc=0.3706
iter 7: train loss/sent=0.1843, time=0.59s
iter 7: test acc=0.3756
iter 8: train loss/sent=0.1388, time=0.60s
iter 8: test acc=0.3647
iter 9: train loss/sent=0.1024, time=0.56s
iter 9: test acc=0.3661
iter 10: train loss/sent=0.0817, time=0.59s
iter 10: test acc=0.3620
iter 11: train loss/sent=0.0633, time=0.59s
iter 11: test acc=0.3629
iter 12: train loss/sent=0.0531, time=0.57s
iter 12: test acc=0.3557
iter 13: train loss/sent=0.0412, time=0.57s
iter 13: test acc=0.3584
iter 14: train loss/sent=0.0364, time=0.59s
iter 14: t